In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

from selective_search import selective_search
import skimage.io

from rcnn import *
from nms import *
from vis import *

Using TensorFlow backend.


In [ ]:
MAX_CATEGORIES = 100

def prune_boxes(boxes, scores):
    print("Prune boxes... ", end="")
    
    # Find scores > 0
    pos_scores = np.argwhere(scores > 0)
    categories = np.unique(pos_scores[:, 1])
    
    # If too many categories, choose the N categories with the highest scores
    if len(categories) > MAX_CATEGORIES:
        best_scores = max(scores[:, categories])
        best_scores = [(score, category) for score, category in zip(best_scores, categories)]
        best_scores.sort(reverse=True)
        categories = [category for _, category in best_scores[:MAX_CATEGORIES]]
    
    idx = 0
    best_boxes, category_ids, best_scores = [], [], []
    for category in categories:
        idxs = pos_scores[1][pos_scores[1] == category]
        category_scores = scores[pos_scores[0][idxs], category]
        scored_boxes = [(box, score) for box, score in zip(boxes[idx], category_scores)]
        
        keep = non_max_suppression(scored_boxes, 0.1)
        best_boxes += [box for box in scored_boxes[keep]]
        category_ids += [category for _ in keep]
        best_scores += category_scores[keep]
        
    keep = non_max_suppression(best_boxes, 0.4)
    category_ids = category_ids[keep]
    best_boxes = best_boxes[keep]
    best_scores = best_scores[keep]
    idxs = [i for i, (_, score) in enumerate(best_boxes) if score >= 1]
    
    if len(idxs) > 1:
        best_boxes = best_boxes[idxs]
        category_ids = category_ids[idxs]
        best_scores = best_scores[idxs]
        
    print("Done.")
    return best_boxes, category_ids, best_scores

In [3]:
def extract_boxes(img, n):
    """
    Uses Selective Search to extract the top N boxes from the image
    
    Args:
        img (ndarray): The image to extract boxes from
        n (int): The maximum number of boxes to return
    Returns:
        The extracted list of boxes
    """
    # Modes = "single", "fast", "quality"
    print("Extracting boxes... ", end="")
    boxes = selective_search(img, mode="fast", random=False)
    print("Done.")

    # Only return first N boxes
    return boxes[:n]

In [4]:
def per_region_features(model, features, img, boxes):
    """
    
    Args:
        model (Model): The model to use
        features ():
        img (ndarray): The image to extract features from
        boxes (list of tuple): The (x1, y1, x2, y2) boxes
    Returns:
        -
    """
    print("Extracting features from regions... ", end="")
    
    
    print("Done.")
    return features

In [ ]:
def prune_boxes_hedging(boxes, scores, categories):
    pass

In [5]:
# Load model
model = load_model("../models/model.pkl")

# Get boxes
img = skimage.io.imread("../data/ex_img.jpg")
num_boxes = 1024
boxes = extract_boxes(img, num_boxes)

# Extract per region features and scores
scores = per_region_features(model, feat, img, boxes)

# Prune boxes based on scores
boxes, category_ids = prune_boxes(boxes, scores)
found_categories = model.classes[category_ids]

# Show detections
m = min(5, len(category_ids))
show_detections(img, boxes[:m], found_categories[:m], category_ids[:m])

Extracting boxes... Done.
